# Self Attention

In [1]:
import tensorflow as tf
from tensorflow.keras import Input, layers

## Calculate attention score $\alpha$ or attention weights

In [19]:

d_model = 3
length = 21

# Calculate wq. wk, wv
wq = layers.Dense(d_model)
wk = layers.Dense(d_model)
wv = layers.Dense(d_model)
input = Input(shape=(length,d_model,))

q = wq(input) #shape = [None,length, d_model]
k = wk(input) #shape = [None,length, d_model]
v = wv(input) #shape = [None,length, d_model]

matmul_qk = tf.matmul(q,k, transpose_b=True) #Shape = [None, length, length]

#scale matmul_qk
#dk is the last dimention of k
dk = tf.cast(tf.shape(k)[-1], tf.float32)
scaled_attention_logits = matmul_qk / tf.math.sqrt(dk) #Shape = [N, length,length]

## If you have masking , here it is!

- padding mask
- look ahead mask

In [ ]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    # add extra dimensions to add the padding
    # # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


In [ ]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)


In [3]:
mask = None
if mask is not None:
    scaled_attention_logits += (mask * -1e9)


## Calculate QKV or self attention feature map

In [23]:
#Softmax Here attention weight also called attention score
attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1) #Shape = [N, length, length]

#calculate alpha
output = tf.matmul(attention_weights, v) #shape = [None, length, d_model]